In [8]:
import yfinance as yf

# Choose an interval: '1d' (daily), '1wk' (weekly), '1mo' (monthly), etc.
interval = '1mo'  # Change this to '1wk' or '1mo' as needed

# Download historical data for S&P 500 (^GSPC) with the chosen interval
sp500 = yf.download("^GSPC", period="1y", interval=interval)

print(f"Downloaded S&P 500 data with interval: {interval}")
print(sp500)


/tmp/ipykernel_12285/2287954889.py:7: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Downloaded S&P 500 data with interval: 1mo
Price             Close         High          Low         Open        Volume
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC         ^GSPC
Date                                                                        
2024-12-01  5881.629883  6099.970215  5832.299805  6040.109863   86064900000
2025-01-01  6040.529785  6128.180176  5773.310059  5903.259766   88639380000
2025-02-01  5954.500000  6147.430176  5837.660156  5969.649902   92317000000
2025-03-01  5611.850098  5986.089844  5488.729980  5968.330078  111387270000
2025-04-01  5569.060059  5695.310059  4835.040039  5597.529785  118936380000
2025-05-01  5911.689941  5968.609863  5578.640137  5625.140137  105346260000
2025-06-01  6204.950195  6215.080078  5861.430176  5896.680176  106456300000
2025-07-01  6339.390137  6427.020020  6177.970215  6187.250000  114004890000
2025-08-01  6460.259766  6508.229980  6212.689941  6287.279785   99352030000
2025-09-01  6688.459961  6699.520

In [2]:
import plotly.graph_objects as go

# Extract OHLC data from the MultiIndex DataFrame
# The data has columns like ('Close', '^GSPC'), ('High', '^GSPC'), etc.
ohlc_data = sp500[['Open', 'High', 'Low', 'Close']]

# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=ohlc_data.index,
    open=ohlc_data[('Open', '^GSPC')],
    high=ohlc_data[('High', '^GSPC')],
    low=ohlc_data[('Low', '^GSPC')],
    close=ohlc_data[('Close', '^GSPC')],
    name='S&P 500'
)])

# Update layout
fig.update_layout(
    title='S&P 500 Candlestick Chart (1 Year)',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,  # Hide the range slider for cleaner look
    height=600
)

# Show the chart
fig.show()
